In [5]:
from sqlalchemy import create_engine
import pandas as pd
from config import server, database, driver 

connection_string = f"mssql+pyodbc://@{server}/{database}?driver={driver}"

engine = create_engine(connection_string)


When is the peak season of our ecommerce ?

In [6]:

query = '''
    select 
        d.year
        ,d.quarter
        ,count(o.order_date_key) total_orders
        ,sum(o.shipping_cost + o.price) total_revenue
    from order_fact o
    join date_dim d
    on o.order_date_key = d.date_key
    group by d.quarter, d.year
    order by total_revenue desc
'''
PeakSeason_Quarter = pd.read_sql(query, con=engine)
PeakSeason_Quarter.head()

,year,quarter,total_orders,total_revenue
0,2018,2,25149,3.606614e+09
1,2018,1,26579,3.532260e+09
2,2017,4,22526,3.062347e+09
3,2017,3,16463,2.216927e+09
4,2018,3,13266,1.876470e+09


In [7]:
query = '''
    select 
        d.year
        ,d.month
        ,count(o.order_date_key) total_orders
        ,sum(o.shipping_cost + o.price) total_revenue
    from order_fact o
    join date_dim d
    on o.order_date_key = d.date_key
    group by d.month, d.year
    order by total_revenue desc
'''
PeakSeason_Month = pd.read_sql(query, con=engine)
PeakSeason_Month.head()

,year,month,total_orders,total_revenue
0,2017,11,9554,1.282683e+09
1,2018,5,8677,1.242632e+09
2,2018,4,8682,1.242087e+09
3,2018,3,8969,1.234646e+09
4,2018,1,9025,1.203787e+09


What time users are most likely make an order or using the ecommerce app?

In [20]:
query = ''' 

    select 
        hour + 1 hour
        ,count(*) orders_count
    from order_fact o
    join time_dim t
    on o.order_time_key = t.time_key
    group by t.hour
    order by orders_count desc
'''

MostOrderingTime = pd.read_sql(query, con=engine)
MostOrderingTime.head()

,hour,orders_count
0,15,8346
1,17,8192
2,12,7998
3,14,7952
4,16,7890


What is the preferred way to pay in the ecommerce?

In [8]:
query = '''
    select top 1
        payment_type
        ,count(*) count
    from payment_dim
    group by payment_type
    order by count desc
'''

MostPopularPayment = pd.read_sql(query, con=engine)
MostPopularPayment['payment_type'][0]


'credit_card'

How many installment is usually done when paying in the ecommerce?

In [9]:
query = '''
    select 
        avg(payment_installments) avg_payment_installments
    from payment_dim;
'''
AVGPaymentInstallments = pd.read_sql(query, con=engine)
AVGPaymentInstallments['avg_payment_installments'][0]

2

What is the average spending time for user for our ecommerce?

In [10]:
query = '''
    select
        avg(datediff(day , do.date,dd.date)) avg_time
    from order_fact o
    join date_dim do
    on o.order_date_key = do.date_key
    join date_dim dd
    on o.delivered_date_key = dd.date_key
    where o.order_status = 'delivered'
'''
AVGUserSpendingTime = pd.read_sql(query, con=engine)
AVGUserSpendingTime['avg_time'][0]

12

What is the frequency of purchase on each state?

In [11]:
query = '''

    select 
        s.seller_state
        ,count(o.order_key) count
    from order_fact o
    join seller_dim s
    on o.seller_key = s.seller_key
    group by s.seller_state
    order by count desc;
    
'''

PurchaseFrequencyOfState = pd.read_sql(query, con=engine)
PurchaseFrequencyOfState.head(10)

,seller_state,count
0,BANTEN,36545
1,JAWA BARAT,12024
2,JAWA TENGAH,11237
3,KALIMANTAN TIMUR,10338
4,JAWA TIMUR,9006
5,DKI JAKARTA,8308
6,SUMATERA UTARA,6315
7,SULAWESI SELATAN,4606
8,RIAU,3112
9,LAMPUNG,2390


Which logistic route that have heavy traffic in our ecommerce?

In [12]:
query = '''
    select 
        s.seller_state
        ,s.seller_city
        ,u.user_state
        ,u.user_city
        ,count(o.order_key) count
    from order_fact o
    join seller_dim s
    on o.seller_key = s.seller_key
    join user_dim u
    on o.user_key = u.user_key
    group by u.user_city, u.user_state, s.seller_city, s.seller_state
    order by count desc;
'''

RoutesTraffic = pd.read_sql(query, con=engine)
RoutesTraffic.head(10)

,seller_state,seller_city,user_state,user_city,count
0,BANTEN,KOTA TANGERANG,BANTEN,KOTA TANGERANG,5328
1,BANTEN,KOTA TANGERANG,DKI JAKARTA,KOTA JAKARTA BARAT,1773
2,KALIMANTAN TIMUR,KABUPATEN BERAU,BANTEN,KOTA TANGERANG,1621
3,BANTEN,KOTA TANGERANG,BANTEN,KABUPATEN TANGERANG,769
4,KALIMANTAN TIMUR,KABUPATEN BERAU,DKI JAKARTA,KOTA JAKARTA BARAT,705
5,BANTEN,KOTA TANGERANG,JAWA BARAT,KABUPATEN BEKASI,560
6,JAWA BARAT,KABUPATEN BOGOR,BANTEN,KOTA TANGERANG,532
7,BANTEN,KOTA TANGERANG,DKI JAKARTA,KOTA JAKARTA SELATAN,469
8,RIAU,KOTA PEKANBARU,BANTEN,KOTA TANGERANG,460
9,BANTEN,KOTA TANGERANG,DKI JAKARTA,KOTA JAKARTA TIMUR,459


In [13]:
query = '''
    select 
        s.seller_state
        ,u.user_state
        ,count(o.order_key) count
    from order_fact o
    join seller_dim s
    on o.seller_key = s.seller_key
    join user_dim u
    on o.user_key = u.user_key
    group by u.user_state, s.seller_state
    order by count desc;
'''

StatesTraffic = pd.read_sql(query, con=engine)
StatesTraffic.head(10)

,seller_state,user_state,count
0,BANTEN,BANTEN,8296
1,BANTEN,JAWA BARAT,4800
2,BANTEN,DKI JAKARTA,4241
3,BANTEN,JAWA TIMUR,3270
4,BANTEN,JAWA TENGAH,3150
5,JAWA BARAT,BANTEN,2677
6,JAWA TENGAH,BANTEN,2511
7,KALIMANTAN TIMUR,BANTEN,2339
8,JAWA TIMUR,BANTEN,1945
9,JAWA BARAT,JAWA BARAT,1721


How many late delivered order in our ecommerce? Are late order affecting the customer satisfaction?

In [14]:
query = '''

    select 
        'Late' orders_type
        ,count(o.order_key) orders_count
        ,avg(feedback_score) avg_score
    from order_fact o
    join date_dim dd on o.delivered_date_key = dd.date_key
    join date_dim ed on o.estimated_time_delivery_key = ed.date_key
    left join feedback_dim f on o.order_key = f.order_key
    where dd.date > ed.date
    and f.feedback_score is not null
    union 
    select 
        'On Time' orders_type
        ,count(o.order_key) orders_count
        ,avg(feedback_score) avg_score
    from order_fact o
    join date_dim dd on o.delivered_date_key = dd.date_key
    join date_dim ed on o.estimated_time_delivery_key = ed.date_key
    left join feedback_dim f on o.order_key = f.order_key
    where dd.date <= ed.date
    and f.feedback_score is not null;

'''

OnTime_Late_Delivery = pd.read_sql(query, con=engine)
OnTime_Late_Delivery.head()

,orders_type,orders_count,avg_score
0,Late,6474,2
1,On Time,88055,4


How long are the delay for delivery / shipping process in each state?

In [15]:
query = ''' 
    select 
        s.seller_state [from]
        ,u.user_state [to]
        ,count(o.order_key) total_orders
        ,avg(datediff(day, ed.date, dd.date)) avg_delivery_days
    from order_fact o
    join seller_dim s on o.seller_key = s.seller_key
    join user_dim u on o.user_key = u.user_key
    join date_dim dd on o.delivered_date_key = dd.date_key
    join date_dim ed on o.estimated_time_delivery_key = ed.date_key
    where dd.date > ed.date
    group by u.user_state, s.seller_state
    order by avg_delivery_days desc
'''

LateDeliveryRoutes = pd.read_sql(query, con=engine)
LateDeliveryRoutes.head(10)

,from,to,total_orders,avg_delivery_days
0,ACEH,JAMBI,1,162
1,NUSA TENGGARA BARAT,DI YOGYAKARTA,1,96
2,BALI,KALIMANTAN SELATAN,1,94
3,JAWA TENGAH,KEPULAUAN BANGKA BELITUNG,2,87
4,KALIMANTAN TIMUR,SULAWESI BARAT,4,82
5,KALIMANTAN TENGAH,RIAU,1,76
6,SULAWESI BARAT,JAWA BARAT,1,68
7,PAPUA,RIAU,2,63
8,BALI,KEPULAUAN BANGKA BELITUNG,1,59
9,SULAWESI UTARA,KALIMANTAN TENGAH,2,58


In [16]:
query = ''' 

    select 
        u.user_state 
        ,count(o.order_key) total_orders
        ,avg(datediff(day, ed.date, dd.date)) avg_delivery_days
    from order_fact o
    join user_dim u on o.user_key = u.user_key
    join date_dim dd on o.delivered_date_key = dd.date_key
    join date_dim ed on o.estimated_time_delivery_key = ed.date_key
    where dd.date > ed.date
    group by u.user_state
    order by avg_delivery_days desc
    
'''

LateDeliveryStates = pd.read_sql(query, con=engine)
LateDeliveryStates.head(10)

,user_state,total_orders,avg_delivery_days
0,BENGKULU,22,20
1,NUSA TENGGARA BARAT,22,16
2,SULAWESI BARAT,34,16
3,KEPULAUAN BANGKA BELITUNG,31,15
4,KEPULAUAN RIAU,70,14
5,BALI,170,13
6,JAMBI,86,13
7,SULAWESI TENGGARA,68,13
8,SULAWESI UTARA,105,13
9,SUMATERA SELATAN,208,11


How long are the difference between estimated delivery time and actual delivery time in each state?

In [17]:
query = '''

    select 
        u.user_state as state,
        avg(datediff(day, ed.date, dd.date)) as avg_delivery_diff,
        max(datediff(day, ed.date, dd.date)) as max_delivery_diff,
        min(datediff(day, ed.date, dd.date)) as min_delivery_diff,
        count(o.order_key) as total_orders
    from order_fact o
    join user_dim u on o.user_key = u.user_key
    join date_dim dd on o.delivered_date_key = dd.date_key
    join date_dim ed on o.estimated_time_delivery_key = ed.date_key
    group by u.user_state
    order by avg_delivery_diff desc

'''

DeliveryTimeByState = pd.read_sql(query, con=engine)
DeliveryTimeByState.head(10)


,state,avg_delivery_diff,max_delivery_diff,min_delivery_diff,total_orders
0,BALI,-11,69,-52,1682
1,BANTEN,-11,165,-78,26785
2,DI YOGYAKARTA,-11,105,-49,2187
3,JAMBI,-11,162,-42,1418
4,JAWA BARAT,-11,167,-84,15680
5,JAWA TENGAH,-11,161,-68,10448
6,JAWA TIMUR,-11,153,-147,10235
7,SULAWESI TENGAH,-11,45,-66,1172
8,SULAWESI TENGGARA,-12,52,-54,1059
9,SULAWESI UTARA,-12,155,-77,1462
